In [34]:
import pandas as pd
import json

In [35]:
file_path = "Dataset.json"
with open(file_path, 'r') as file:
    json_data = json.load(file)

transactions = json_data['FinancialSummary'][0]['Transactions']['Transaction'][1:]

# dataframe with 1st row as header
df = pd.DataFrame(transactions, columns=transactions[0].keys())
df.head()

,type,mode,amount,currentBalance,transactionTimestamp,valueDate,txnId,narration,reference
0,DEBIT,OTHERS,500.0,15389.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,UPI-KULDEEP S O CHATTAR -KULDEEP9650@YBL -PUNB...,
1,DEBIT,OTHERS,200.0,15189.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,UPI-LATA-PAYTM-73272027@PAYTM-PYTM0123456-2323...,
2,DEBIT,OTHERS,150.0,15039.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,UPI-RG SONS AND OTHERS L-PAYTM-70153610@ PAYTM...,
3,DEBIT,OTHERS,150.0,14889.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,UPI-VIJAY KUMAR-PAYTM-62448237@PAYTM-PYT M0123...,
4,DEBIT,OTHERS,3.0,14886.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,UPI-LAXMI DAS-PAYTM-70579281@PAYTM-PYTM0123456...,


In [36]:
import nltk
df['narration'] = df['narration'].str.lower()

In [37]:
import ocr

# loop through the narration value for each row and process text and get category
for index, row in df.iterrows():
    concat_output = row['narration']
    filtered_list = ocr.process_text(concat_output)
    category = ocr.get_category(filtered_list)
    df.at[index, 'category'] = category

df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package st

,type,mode,amount,currentBalance,transactionTimestamp,valueDate,txnId,narration,reference,category
0,DEBIT,OTHERS,500.0,15389.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,upi-kuldeep s o chattar -kuldeep9650@ybl -punb...,,others
1,DEBIT,OTHERS,200.0,15189.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,upi-lata-paytm-73272027@paytm-pytm0123456-2323...,,others
2,DEBIT,OTHERS,150.0,15039.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,upi-rg sons and others l-paytm-70153610@ paytm...,,others
3,DEBIT,OTHERS,150.0,14889.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,upi-vijay kumar-paytm-62448237@paytm-pyt m0123...,,others
4,DEBIT,OTHERS,3.0,14886.36,2022-11-19T00:00:00.000+00:00,2022-11-19,dummy,upi-laxmi das-paytm-70579281@paytm-pytm0123456...,,others


In [38]:
# create a new dataframe with columns as category, type, amount, timestamp, narration, initBal, mode, billImage
df2 = pd.DataFrame(columns=['category', 'type', 'amount', 'timestamp', 'narration', 'initialBalance', 'mode', 'billImage'])

In [39]:
df2['category'] = df['category']
df2['type'] = df['type']
df2['amount'] = df['amount']
df2['narration'] = df['narration']
df2['mode'] = df['mode']
df2['billImage'] = ""
df2['timestamp'] = df['transactionTimestamp']
# if Debit then init Balance is Current Balance - Amount
# if Credit then init Balance is Current Balance + Amount
# loop through the dataframe and update initBal column
for index, row in df.iterrows():
    if row['type'] == 'Debit':
        df2.at[index, 'initialBalance'] = float(row['currentBalance']) - float(row['amount'])
    else:
        df2.at[index, 'initialBalance'] = float(row['currentBalance']) + float(row['amount'])

In [40]:
df2.head()

,category,type,amount,timestamp,narration,initialBalance,mode,billImage
0,others,DEBIT,500.0,2022-11-19T00:00:00.000+00:00,upi-kuldeep s o chattar -kuldeep9650@ybl -punb...,15889.36,OTHERS,
1,others,DEBIT,200.0,2022-11-19T00:00:00.000+00:00,upi-lata-paytm-73272027@paytm-pytm0123456-2323...,15389.36,OTHERS,
2,others,DEBIT,150.0,2022-11-19T00:00:00.000+00:00,upi-rg sons and others l-paytm-70153610@ paytm...,15189.36,OTHERS,
3,others,DEBIT,150.0,2022-11-19T00:00:00.000+00:00,upi-vijay kumar-paytm-62448237@paytm-pyt m0123...,15039.36,OTHERS,
4,others,DEBIT,3.0,2022-11-19T00:00:00.000+00:00,upi-laxmi das-paytm-70579281@paytm-pytm0123456...,14889.36,OTHERS,


In [41]:
df['category'].value_counts()

others          28
home_utility     1
Name: category, dtype: int64

In [42]:
df['mode'].value_counts()

OTHERS    29
Name: mode, dtype: int64

In [43]:
df['type'].value_counts()

DEBIT     25
CREDIT     4
Name: type, dtype: int64

In [44]:
df2.to_csv('Dataset.csv', index=False)